In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [2]:
COLUMN_NAMES = [
    'user',
    'activity',
    'timestamp',
    'x-axis',
    'y-axis',
    'z-axis'
]

LABELS = [
    'Downstairs',
    'Jogging',
    'Sitting',
    'Standing',
    'Upstairs',
    'Walking'
]

In [3]:
path = '../../DDC_Data/wisdm/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt'

In [4]:
data = pd.read_csv(path, header=None, names=COLUMN_NAMES)
data['z-axis'].replace({';': ''}, regex=True, inplace=True)
data = data.dropna()

In [5]:
data 

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
5,33,Jogging,49106442306000,1.375655,-2.492524,-6.510526
6,33,Jogging,49106542312000,-0.612916,10.569390,5.706926
7,33,Jogging,49106652389000,-0.503953,13.947236,7.0553403
8,33,Jogging,49106762313000,-8.430995,11.413852,5.134871
9,33,Jogging,49106872299000,0.953424,1.375655,1.6480621


In [6]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import os
import math

In [8]:
data_convoluted = []
labels = []

    # Slide a "SEGMENT_TIME_SIZE" wide window with a step size of "TIME_STEP"
for i in range(0, len(data)):
    x = data['x-axis'][i]
    y = data['y-axis'][i]
    z = data['z-axis'][i]
    data_convoluted.append(x)
    data_convoluted.append(y)
    data_convoluted.append(z)

    # Label for a data window is the label that appears most commonly
    label = data['activity'][i]
    labels.append(label)

KeyError: 343416

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_convoluted, labels, test_size=0.3)

In [ ]:
classifier=svm.SVC()

In [ ]:
parameters=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 
             'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
model=GridSearchCV(classifier,parameters,n_jobs=-1,cv=4,verbose=4)
model.fit(x_train.as_matrix(),y_train.as_matrix().ravel().T)

In [ ]:
from sklearn.metrics import accuracy_score
ypred=model.predict(x_test)
accuracy=accuracy_score(y_test,ypred)

In [ ]:
print ('Best Parameters: '+ str(model.best_params_))
print ('Accuracy Score: '+ str(accuracy*100) + ' %')

In [ ]:
%run eval_score.ipynb 

In [ ]:
show_conf_matrix(y_test, ypred, LABELS)
show_clf_report(y_test, ypred, LABELS)